In [1]:
!pip install nltk rouge-score bert-score

  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.8 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=7e591113fdd23ffcf27890d580485845061a665be286b909ff8abf54085f8644
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
import numpy as np
import pandas as pd
import nltk
import os
import spacy
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
from bert_score import score
from transformers import pipeline

2024-02-03 19:20:29.352669: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-03 19:20:29.352840: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-03 19:20:29.522744: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", message="Some weights of *")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
from datasets import load_dataset
dataset = load_dataset("opinosis")
dataset

Generating train split:   0%|          | 0/51 [00:00<?, ? examples/s]

Dataset opinosis downloaded and prepared to /root/.cache/huggingface/datasets/opinosis/default/1.0.0/1852a0faaef01fdbc6149f3f6c8a0c405fe27aa050e691be443f86e4e1273417. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['review_sents', 'summaries'],
        num_rows: 51
    })
})

In [5]:
data = dataset['train'].to_pandas()
data.head()

,review_sents,summaries
0,", and is very, very accurate .\r\n but for the...",[This unit is generally quite accurate. \r\nS...
1,"The room was not overly big, but clean and ve...",[The rooms were not large but were clean and v...
2,After I plugged it in to my USB hub on my com...,[Battery life is exceptional.\r\nThe Kindle ca...
3,short battery life I moved up from an 8gb .\...,[The battery life is too short.\r\nThe time be...
4,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh ...",[The battery life is longer then 5 hours.\r\nB...


In [6]:
data['summaries'][0]

array(['This unit is generally quite accurate.  \r\nSet-up and usage are considered to be very easy. \r\nThe maps can be updated, and tend to be reliable.',
       "The Garmin seems to be generally very accurate.\r\nIt's easy to use with an intuitive interface.",
       'It is very accurate, even in destination time.',
       'Very accurate with travel and destination time.\r\nNegatives are not accurate with speed limits and rural roads.',
       'Its accurate, fast and its simple operations make this a for sure buy.'],
      dtype=object)

In [7]:
summaries = []
for i in range(len(data)):
    summaries.append(data['summaries'][i][0])

In [8]:
data = data.drop(['summaries'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   review_sents  51 non-null     object
dtypes: object(1)
memory usage: 536.0+ bytes


In [9]:
data['summaries'] = summaries
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   review_sents  51 non-null     object
 1   summaries     51 non-null     object
dtypes: object(2)
memory usage: 944.0+ bytes


In [10]:
data['review_sents'].isnull().sum()

0

In [11]:
def calculate_redundancy(summaries):
    
    total_tokens = sum(len(summary.split()) for summary in summaries)
    unique_tokens = len(set(token for summary in summaries for token in summary.split()))
    redundancy_score = 1 - (unique_tokens / total_tokens)
    
    return redundancy_score

In [12]:
def calculate_bleu(actual_summary, predicted_summary):
    actual_summary_tokenized = [[ref.split()] for ref in actual_summary]         
    pred_summary_tokenized = [output.split() for output in predicted_summary]         
    bleu_score = corpus_bleu(actual_summary_tokenized, pred_summary_tokenized)
    return bleu_score

In [13]:
def calculate_bert(actual_summary, predicted_summary):
    P, R, F1 = score(actual_summary, predicted_summary, lang='en', verbose=False)     # returns Precision, Recall and F1 score
    bert_score = F1.mean().item()  
    return bert_score

In [14]:
def calculate_rouge(actual_summary, predicted_summary):
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)         
             
    rouge1_scores = []
    rouge2_scores = []
    for pred, actual in zip(predicted_summary, actual_summary):
        rouge_scores = rouge.score(pred, actual)                                               
        rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
        rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
            
    rouge1 = sum(rouge1_scores) / len(rouge1_scores)                                      
    rouge2 = sum(rouge2_scores) / len(rouge2_scores)
        
    return rouge1, rouge2

In [15]:
def named_entity_recognition(original_texts, hypotheses):
    nlp = spacy.load("en_core_web_sm")
    precision_sum, recall_sum = 0, 0
    
    for original_text, hypothesis in zip(original_texts, hypotheses):
#         print(type(original_text))
        original_doc = nlp(original_text)
        hypothesis_doc = nlp(hypothesis)
        
        original_entities = set([ent.text for ent in original_doc.ents])
        hypothesis_entities = set([ent.text for ent in hypothesis_doc.ents])
        
        common_entities = original_entities.intersection(hypothesis_entities)
        
        precision_sum += len(common_entities) / len(hypothesis_entities) if len(hypothesis_entities) > 0 else 0
        recall_sum += len(common_entities) / len(original_entities) if len(original_entities) > 0 else 0
    
    num_samples = len(original_texts)
    avg_precision = precision_sum / num_samples
    avg_recall = recall_sum / num_samples
    
    return avg_precision, avg_recall


In [16]:
# named_entity_recognition(df['review_sents'],df['summaries'])

In [17]:
def get_metric(original, predicted, actual):
    redundancy = calculate_redundancy(predicted)
    rouge1, rouge2 = calculate_rouge(actual, predicted)
    bert = calculate_bert(actual, predicted)
    bleu = calculate_bleu(actual, predicted)
    precision, recall = named_entity_recognition(original, predicted)
    
    return redundancy, bleu, bert, rouge1, rouge2, precision, recall

In [18]:
df = data.sample(n=15, replace=False).reset_index(drop=True)
df = df.dropna()
df.head()

,review_sents,summaries
0,A few other things I'd like to point out is t...,The size is great and allows for excellent por...
1,Being able to change the font sizes is awesome...,Font size is easily adjusted depending on your...
2,"Good Value good location , ideal choice .\r\n...",Hotel is located nicely in close proximity to ...
3,"6GHz 533FSB cpu, glossy display, 3, Cell 23Wh ...",The battery life is longer then 5 hours.\r\nBu...
4,Great location for tube and we crammed in a f...,"Location is excellent, very close to the Glouc..."


In [19]:
df['review_sents'].isnull().sum()

0

In [20]:
df['review_sents']

0      A few other things I'd like to point out is t...
1     Being able to change the font sizes is awesome...
2     Good Value good location ,  ideal choice .\r\n...
3     6GHz 533FSB cpu, glossy display, 3, Cell 23Wh ...
4      Great location for tube and we crammed in a f...
5      The room was not overly big, but clean and ve...
6      Mediocre room and service for a very extravag...
7      In fact, the entire navigation structure has ...
8     Another feature on the 255w is a display of th...
9      After slowing down, transmission has to be  k...
10    not customer, oriented hotelvery low service l...
11     As always, the video screen is sharp and brig...
12    All in all, a normal chain hotel on a nice loc...
13    Windows 7 is quite simply faster, more stable,...
14    3 quot  widescreen display was a bonus .\r\n T...
Name: review_sents, dtype: object

In [21]:
models = ["facebook/bart-large-cnn", 
          "philschmid/bart-large-cnn-samsum", 
          "google/pegasus-cnn_dailymail", 
          "google/pegasus-large"
         ]

In [22]:
# pipe = pipeline("summarization", model = "google/pegasus-large")
# pipe(df['review_sents'][0], max_length=130, min_length=30, truncation= True)[0]['summary_text']

In [23]:
redundancy_score=[]
bleu_score=[]
bert_score=[]
rouge1_score=[]
rouge2_score=[]
named_entity_precision=[]
named_entity_recall=[]

**Model 1: facebook/bart-large-cnn**

In [24]:
pipe = pipeline("summarization", model = "facebook/bart-large-cnn")
summary = []
for j in range(0,len(df)):
    pred = pipe(df['review_sents'][j], max_length=130, min_length=30, truncation= True)[0]['summary_text']
    summary.append(pred)
print(summary[0])

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

The netbook is a full featured laptop squeezed into a smaller and manageable size. The keyboards are 92% the size of those found on a standard computer and the machines run a little hot. The keyboard has a responsive feel and the larger keys   along with the oversize right shift key are a real plus.


In [25]:
redundancy,bleu,bert,rouge1,rouge2,precision,recall = get_metric(df['review_sents'], summary, df['summaries'].tolist())

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
print('PARAMETERS')
print('Redundancy Score: ',redundancy)
print('BLEU Score: ',bleu)
print('BERT Score: ',bert)
print('Rouge-1 Score: ',rouge1)
print('Rouge-2 Score: ',rouge2)
print('Named Entity Precision: ',precision)
print('Named Entity Recall: ',recall)

PARAMETERS
Redundancy Score:  0.4483146067415731
BLEU Score:  0.036456255427391254
BERT Score:  0.8533966541290283
Rouge-1 Score:  0.19847826791798448
Rouge-2 Score:  0.04086700195045553
Named Entity Precision:  0.8666666666666667
Named Entity Recall:  0.04578035332582495


In [27]:
redundancy_score.append(redundancy)
bleu_score.append(bleu)
bert_score.append(bert)
rouge1_score.append(rouge1)
rouge2_score.append(rouge2)
named_entity_precision.append(precision)
named_entity_recall.append(recall)

**Model 2: philschmid/bart-large-cnn-samsum**

In [28]:
pipe = pipeline("summarization", model = "philschmid/bart-large-cnn-samsum")
summary = []
for j in range(0,len(df)):
    pred = pipe(df['review_sents'][j], max_length=130, min_length=30, truncation= True)[0]['summary_text']
    summary.append(pred)
print(summary[0])

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Pro's 10-inch laptop has a full keyboard with full size keys 92% the size of those found on a standard computer. The keyboard has a responsive feel and the larger keys and the oversize right shift key are a real plus. It's the perfect size and weight to take almost anywhere. The performance is great for it's size and the battery life is very good. The only complaint is the compressed size of the UP and DOWN arrows.


In [29]:
redundancy,bleu,bert,rouge1,rouge2,precision,recall = get_metric(df['review_sents'], summary, df['summaries'].tolist())
print('PARAMETERS')
print('Redundancy Score: ',redundancy)
print('BLEU Score: ',bleu)
print('BERT Score: ',bert)
print('Rouge-1 Score: ',rouge1)
print('Rouge-2 Score: ',rouge2)
print('Named Entity Precision: ',precision)
print('Named Entity Recall: ',recall)

redundancy_score.append(redundancy)
bleu_score.append(bleu)
bert_score.append(bert)
rouge1_score.append(rouge1)
rouge2_score.append(rouge2)
named_entity_precision.append(precision)
named_entity_recall.append(recall)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PARAMETERS
Redundancy Score:  0.4843205574912892
BLEU Score:  0.011776459071924514
BERT Score:  0.8565710783004761
Rouge-1 Score:  0.2087507659928366
Rouge-2 Score:  0.04839622959867704
Named Entity Precision:  0.8082539682539682
Named Entity Recall:  0.0675569756939505


**Model 3: google/pegasus-cnn_dailymail**

In [30]:
pipe = pipeline("summarization", model = "google/pegasus-cnn_dailymail")
summary = []
for j in range(0,len(df)):
    pred = pipe(df['review_sents'][j], max_length=130, min_length=30, truncation= True)[0]['summary_text']
    summary.append(pred)
print(summary[0])

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Small, easy to grab and go, and can do your business in places that wouldn't open a full size laptop .<n>Full sized laptops are so freaking bulky, heavy, and the battery wouldnt even make it though one of my evening classes .


In [31]:
redundancy,bleu,bert,rouge1,rouge2,precision,recall = get_metric(df['review_sents'], summary, df['summaries'].tolist())
print('PARAMETERS')
print('Redundancy Score: ',redundancy)
print('BLEU Score: ',bleu)
print('BERT Score: ',bert)
print('Rouge-1 Score: ',rouge1)
print('Rouge-2 Score: ',rouge2)
print('Named Entity Precision: ',precision)
print('Named Entity Recall: ',recall)

redundancy_score.append(redundancy)
bleu_score.append(bleu)
bert_score.append(bert)
rouge1_score.append(rouge1)
rouge2_score.append(rouge2)
named_entity_precision.append(precision)
named_entity_recall.append(recall)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PARAMETERS
Redundancy Score:  0.4738955823293173
BLEU Score:  0.03730119405767918
BERT Score:  0.8428364396095276
Rouge-1 Score:  0.18423109242082478
Rouge-2 Score:  0.04431688712384518
Named Entity Precision:  0.6952380952380952
Named Entity Recall:  0.040878784848358524


**Model 4: google/pegasus-large**

In [32]:
pipe = pipeline("summarization", model = "google/pegasus-large")
summary = []
for j in range(0,len(df)):
    pred = pipe(df['review_sents'][j], max_length=130, min_length=30, truncation= True)[0]['summary_text']
    summary.append(pred)
print(summary[0])

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

The machine has the most comfortable small keyboard I've ever used, because the keys are flat and so the surfaces that your fingerpads touch are actually about the size of standard keyboards . It won't give you all the functionality of a full, sized laptop, and the small screen may take some getting used to, but no regular laptop gives you this kind of battery life, and most of the functions absent here aren't used by most people most of the time anyway, so if the tradeoff is fewer features for greatly, increased battery life, I'll take the latter .


In [33]:
redundancy,bleu,bert,rouge1,rouge2,precision,recall = get_metric(df['review_sents'], summary, df['summaries'].tolist())
print('PARAMETERS')
print('Redundancy Score: ',redundancy)
print('BLEU Score: ',bleu)
print('BERT Score: ',bert)
print('Rouge-1 Score: ',rouge1)
print('Rouge-2 Score: ',rouge2)
print('Named Entity Precision: ',precision)
print('Named Entity Recall: ',recall)

redundancy_score.append(redundancy)
bleu_score.append(bleu)
bert_score.append(bert)
rouge1_score.append(rouge1)
rouge2_score.append(rouge2)
named_entity_precision.append(precision)
named_entity_recall.append(recall)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PARAMETERS
Redundancy Score:  0.6032064128256514
BLEU Score:  0.03588999604693392
BERT Score:  0.8325999975204468
Rouge-1 Score:  0.1422359487378335
Rouge-2 Score:  0.031282290793376064
Named Entity Precision:  0.8583333333333333
Named Entity Recall:  0.05954573946137655


# Topsis for Comparison

In [34]:
scores = [redundancy_score, bleu_score, bert_score, rouge1_score, rouge2_score, named_entity_precision, named_entity_recall]
for score in scores:
    for i in range(len(score)):
        score[i]= np.round(score[i],3)

In [35]:
scores

[[0.448, 0.484, 0.474, 0.603],
 [0.036, 0.012, 0.037, 0.036],
 [0.853, 0.857, 0.843, 0.833],
 [0.198, 0.209, 0.184, 0.142],
 [0.041, 0.048, 0.044, 0.031],
 [0.867, 0.808, 0.695, 0.858],
 [0.046, 0.068, 0.041, 0.06]]

In [36]:
topsis_data= pd.DataFrame({
    'Model': models,
    'Redundancy': redundancy_score,
    'BLEU': bleu_score,
    'BERT': bert_score,
    'Rouge-1': rouge1_score,
    'Rouge-2': rouge2_score,
    'Named Entity Precision': named_entity_precision,
    'Named Entity Recall': named_entity_recall
})

In [37]:
weights = [5,2,3,4,4,3,2]
impacts = ['-','+','+','+','+','+','+']

In [38]:
def topsis(data, weight, impact):
    normData = data.copy()
    
    for i, col in enumerate(data.columns[0:]):
        normData[col] = data[col] / np.linalg.norm(data[col])
    normData = normData*weight
    # print(normData)

    ideal_best = np.max(normData, axis=0)
    ideal_worst = np.min(normData, axis=0)
    for idx, x in enumerate(impact):
        if(x=='-'):
            ideal_worst.iloc[idx], ideal_best.iloc[idx] = ideal_best.iloc[idx], ideal_worst.iloc[idx]
    # print(ideal_best)
    # print(ideal_worst)

    separation_best = np.linalg.norm(normData - ideal_best, axis=1)
    separation_worst = np.linalg.norm(normData - ideal_worst, axis=1)

    score = separation_worst / (separation_best + separation_worst)
    res = pd.DataFrame()
    res['TOPSIS Score'] = score
    res['Rank'] = res['TOPSIS Score'].rank(ascending=False)

    return res

In [39]:
score_table = topsis_data.drop('Model',axis=1)
result = topsis(score_table, weights, impacts)
result['Rank'] = result['Rank'].astype(int)
topsis_data['Topsis Score'] = result['TOPSIS Score']
topsis_data['Rank'] = result['Rank']

In [40]:
topsis_data

,Model,Redundancy,BLEU,BERT,Rouge-1,Rouge-2,Named Entity Precision,Named Entity Recall,Topsis Score,Rank
0,facebook/bart-large-cnn,0.448,0.036,0.853,0.198,0.041,0.867,0.046,0.716777,1
1,philschmid/bart-large-cnn-samsum,0.484,0.012,0.857,0.209,0.048,0.808,0.068,0.626148,3
2,google/pegasus-cnn_dailymail,0.474,0.037,0.843,0.184,0.044,0.695,0.041,0.649229,2
3,google/pegasus-large,0.603,0.036,0.833,0.142,0.031,0.858,0.060,0.395293,4
